In [4]:
import  os

In [5]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI


In [6]:
response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(name="source", description="source used to answer the user's question, should be a website.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions

NameError: name 'format_instructions' is not defined

In [ ]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)
prompt

PromptTemplate(input_variables=['question'], partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // answer to the user\'s question\n\t"source": string  // source used to answer the user\'s question, should be a website.\n}\n```'}, template='answer the users question as best as possible.\n{format_instructions}\n{question}')

In [ ]:
PromptTemplate.from_template(
    "answer the users question as best as possible.\n{format_instructions}\n{question}",
    partial_variables={"format_instructions": format_instructions},
)

PromptTemplate(input_variables=['question'], partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // answer to the user\'s question\n\t"source": string  // source used to answer the user\'s question, should be a website.\n}\n```'}, template='answer the users question as best as possible.\n{format_instructions}\n{question}')

In [ ]:
model = OpenAI(temperature=0)
_input = prompt.format_prompt(question="what's the capital of france?")
output = model(_input.to_string())
output_parser.parse(output)


/home/wei/miniconda3/envs/langchain_py310/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/home/wei/miniconda3/envs/langchain_py310/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'answer': 'Paris', 'source': 'https://en.wikipedia.org/wiki/Paris'}

In [ ]:
chain = prompt | model | JsonOutputParser()

In [ ]:
response = chain.invoke("what's the capital of france?")
response

{'answer': 'Paris', 'source': 'https://en.wikipedia.org/wiki/Paris'}

In [ ]:
type(response)

dict

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

llm = ChatOpenAI(temperature=0)

# 定義所需的數據結構, 注意LLM會同時閱讀attribute name、description來進行輸出。
class Joke(BaseModel):
    setup: str = Field(description="问题以设置笑话")
    punchline: str = Field(description="回答以解决笑话")
    year: str = Field(description="today is 2024-1-12, return current year")

parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="回答用户的查询。\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": "Tell me a joke."})

{'setup': "Why couldn't the bicycle find its way home?",
 'punchline': 'Because it lost its bearings!',
 'year': '2024'}

In [ ]:
class Joke(BaseModel):
    setup: str = Field(description="Question to set up the joke")
    punchline: str = Field(description="Answer to resolve the joke")
    year: str = Field(description="today is 2024-1-12, return current year")

parser = JsonOutputParser(pydantic_object=Joke)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"title": "Setup", "description": "Question to set up the joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "Answer to resolve the joke", "type": "string"}, "year": {"title": "Year", "description": "today is 2024-1-12, return current year", "type": "string"}}, "required": ["setup", "punchline", "year"]}
```


In [ ]:
from pydantic import BaseModel
class Ball(BaseModel):
    name: str
    size: int = 5

In [ ]:
ball = Ball(name='hihi', size='a')

ValidationError: 1 validation error for Ball
size
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='a', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing

In [ ]:
type(ball.size)

int

In [ ]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

NameError: name 'functions' is not defined

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")
model = ChatOpenAI()
chain = prompt | model

In [ ]:
functions = [
    {
        "name": "joke",
        "description": "一个笑话",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "笑话的开头"},
                "punchline": {
                    "type": "string",
                    "description": "笑话的结尾",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)
chain.invoke({"foo": "bears"}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"setup":"Why do bears have hairy coats?","punchline":"Because they don\'t like to go to the barber shop!"}', 'name': 'joke'}}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 81, 'total_tokens': 109}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1f77972f-2776-420d-944d-28160c3ac901-0')

In [ ]:
functions = [
    {
        "name": "joke",
        "description": "一个笑话",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "笑话的开头"},
                "punchline": {
                    "type": "string",
                    "description": "笑话的结尾",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]
# chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)
chain = prompt | model
chain.invoke({"foo": "bears"}, config={})

AIMessage(content="Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship any longer!", response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 13, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7843596f-ea0d-4640-966a-8bd42de20ff7-0')

In [ ]:
from operator import itemgetter

In [ ]:
f = itemgetter('foo')

In [ ]:
f({'foso':123, 'bar':456})

KeyError: 'foo'

In [ ]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

In [ ]:
memory = ConversationBufferMemory(return_messages=True)

In [ ]:
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
)

In [ ]:
 his=  RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )

In [ ]:
chain.invoke({'input':123})

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 18, 'total_tokens': 27}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0a01d0bb-5ccf-4076-8f9c-b70bb2f97c0b-0')

In [ ]:
memory.load_memory_variables(inputs=123)

{'history': []}

In [8]:
import logging
logging.getLogger().setLevel(logging.ERROR) # hide warning log


import os


from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)



In [9]:
message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
llm.invoke([message])

AIMessage(content="J'adore la programmation.", response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 19, 'total_tokens': 24}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-2c176ba2-58da-47d5-8913-25fafe56ebe5-0')

In [10]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory

In [11]:
from langchain_community.chat_message_histories import SQLChatMessageHistory


def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

runnable_with_history = RunnableWithMessageHistory(
    llm,
    get_session_history,
)

In [12]:
runnable_with_history.invoke(
    [HumanMessage(content="hi - im wei!")],
    config={"configurable": {"session_id": "5"}},
)

AIMessage(content='Hello Wei! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-3183441a-fc44-4a36-aa02-487c009e8780-0')

In [5]:
runnable_with_history.invoke(
    [HumanMessage(content="who am i?")],
    config={"configurable": {"session_id": "5"}},
)

Error in RootListenersTracer.on_llm_end callback: AttributeError("'str' object has no attribute 'type'")


AIMessage(content="You're someone seeking to understand yourself or perhaps looking for some introspection. Your identity encompasses a wide range of aspects including your experiences, beliefs, values, and relationships. If you have specific details or aspects of your life you're curious about, feel free to share more!", response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 11, 'total_tokens': 64}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_5f4bad809a', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 's

In [2]:
import getpass
import os

# os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

KeyError: 'AZURE_OPENAI_DEPLOYMENT_NAME'